<a href="https://colab.research.google.com/github/karim-mammadov/NLP_MyProjects/blob/main/IMDB_Dataset_of_50K_Movie_Reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile('/content/imdb-dataset-of-50k-movie-reviews.zip','r') as zip_ref:
  zip_ref.extractall('/content')

In [ ]:
import pandas as pd
df = pd.read_csv('/content/IMDB Dataset.csv')

In [ ]:
df.head(5)

In [ ]:
import re

def clean_text(text):
    # HTML tag-ları çıxar
    text = re.sub(r'<.*?>', '', text)
    # Punctuation və xüsusi simvollar (əlavə təmizləmə)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Çoxlu boşluqları 1 boşluğa çevir
    text = re.sub(r'\s+', ' ', text)
    # Aşağı-hərf et
    text = text.lower().strip()
    return text


df['clean_review'] = df['review'].apply(clean_text)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['sentiment'] = le.fit_transform(df['sentiment'])
df.head()

In [ ]:
df.drop('review',axis=1,inplace=True)

# Tokenization

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

texts = [
    "I love NLP",
    "Tokenization is important",
    "This is a sample sentence"
]

#tokenize yarat

tokenizer = Tokenizer(num_words=50, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)

# tokenlere cevir
sequences = tokenizer.texts_to_sequences(texts)
print("Token indexleri:", sequences)

# Pad sequences (hamisini eyni uzunluga getir)

padded = pad_sequences(sequences, maxlen=6, padding='post')
print("Pad edilmis sequences:\n", padded)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [ ]:
# Tokenizer yarat
tokenizer = Tokenizer(num_words=5000, oov_token="‹OOV›")
tokenizer.fit_on_texts(df['clean_review' ])

# Token-lara cevir
sequences = tokenizer.texts_to_sequences(df['clean_review'])

# Pad sequences
max_len = 100 # her cümle 100 token uzunlugunda olacaq
padded = pad_sequences (sequences, maxlen=max_len, padding='post')

# Naticani yoxla
print("avvalca bir cümla: \n", df['clean_review'].iloc[0])

In [ ]:
print("Pad edilmis tokenler:\n", padded[0])

# Embedding

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense



In [ ]:
vocab_size = 5000  #tokenizer.num_words
embedding_dim = 50
max_len = 100

In [ ]:
embedding_layer = Embedding(input_dim = vocab_size, output_dim=embedding_dim)


In [ ]:
import numpy as np

In [ ]:
demo_sequence = np.array([[3, 10, 7, 25, 0, 0]])
embedded_output = embedding_layer(demo_sequence)

print("Shape:", embedded_output.shape)
print("Embedding vector:\n", embedded_output.numpy())
#3D tensor: batch_size x max_len x embedding_dim

In [ ]:
embedding_dim = 50   # her soz ucun 50 olculu dense vector

embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim)

# butun sequences-i embedding-e cevirmek

embedded_output = embedding_layer(padded)  #  shape: (5000, 100, 50)
embedded_output

In [ ]:
# 1-ci cumlenin embedding matrix-i

print("1-ci cumle token sequence:\n", padded[0])
print("Embedding output shape:",  embedded_output[0].shape)
print("Embedding vector ( 5 ilk olcu):\n", embedded_output[0, :5, :5].numpy()) # yalniz ilk 5 token x 5 olcu

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.optimizers import Adam

In [ ]:
model = Sequential([Embedding (input_dim=vocab_size, output_dim=embedding_dim),
                      # embedding layer
                      GlobalAveragePooling1D(),
                      # average pooling
                      Dense(24, activation='relu'),
                      # hidden dense layer
                      Dense (1, activation='sigmoid')
                      # output: 0/1
])

model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(padded, df['sentiment'], epochs=10, validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
x = padded
# token sequences
y = df[ 'sentiment']. values
# 1-ci cümlenin token sequence-i
demo_sequence = x[0]
# padded token sequence
embedded_output = model.layers[0](demo_sequence.reshape(1, -1))
# embedding output
# Convert to numpy
embedded_matrix = embedded_output.numpy()[0] # shape: (max_len, embedding_dim)

# Heatmap (yalni] ilk 20 token va 10 ölçü vizual üçün)
plt.figure(figsize=(10,6))
sns.heatmap(embedded_matrix[:20, :10], annot=True, cmap='viridis', cbar=True)
plt.title("1-ci cümlenin embedding matrix heatmap")
plt.xlabel("Embedding dimensions (first 10)")
plt.ylabel("Tokens (first 20)")
plt. show()